In [13]:
20

20

In [1]:
from federatedscope.register import register_data
from federatedscope.register import register_trainer
from federatedscope.register import register_metric
from federatedscope.register import register_model




### Register model

In [2]:
#from federatedscope.contrib.model.mnist_model import call_my_net
#register_model("mynet", call_my_net)

### Register trainer

In [3]:
from federatedscope.contrib.trainer.laplacian_trainer_with_domain_separation_with_summation_MI import call_laplacian_trainer

register_trainer('laplacian_trainer', call_laplacian_trainer)

/home/michael/Master-Thesis/CKIM_Competition/venv/lib/python3.9/site-packages/torch_geometric/graphgym/imports.py:14: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "
/home/michael/Master-Thesis/CKIM_Competition/venv/lib/python3.9/site-packages/torch_geometric/graphgym/logger.py:23: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "


Device: cuda


### Register metric

### Create the config

#### Set data, model, trainer and metric

In [4]:
from federatedscope.core.configs.config import global_cfg, CN
cfg = global_cfg.clone()

cfg.merge_from_file("scripts/B-FHTL_exp_scripts/Graph-DT/fed_dom_sep_KLD.yaml")
cfg.data.save_dir = 'test_dir'
# cfg.data.type = 'cikm_cup'
#cfg.data.root = 'data'
#cfg.data.shuffle=True
#cfg.data.transform = [['ToTensor'], ['Normalize', {'mean': [0.], 'std': [1]}]]
#cfg.model.type = 'gin'
#cfg.model.out_channels = 10
#cfg.model.hidden = 64
#cfg.model.task='graph'
#cfg.model.dropout = 0.5
#cfg.personalization.local_param = ['encoder_atom', 'encoder', 'clf']#['node_encoder', 'clf']
#cfg.train.batch_or_epoch = "epoch"
cfg.trainer.type = 'laplacian_trainer_with_domain_separation'
cfg.data.batch_size = 64
# cfg.eval.metric = ['mymetric']
cfg.params = CN()
cfg.params.alpha=0.1
cfg.params.csd_importance= 1e2
cfg.params.eps=1e-20
cfg.params.p=0.
cfg.params.diff_importance = 1
cfg.params.mine_lr = 0.02  # MINE lr
cfg.params.lam = 1


#### configure other options

In [5]:
#cfg.use_gpu = True 0.05 [0.02-0.03] 0.04-0.06
#cfg.best_res_update_round_wise_key = "test_loss"

#cfg.federate.mode = 'standalone'

cfg.federate.method = \
    f'Laplacian_domain_sep_sum_MI_csd{cfg.params.csd_importance}_lam{cfg.params.lam}_diff{cfg.params.diff_importance}_MI_lr_{cfg.params.mine_lr}_no_finetune'

#cfg.federate.local_update_steps = 20000000
cfg.personalization.local_update_steps = 20000000
#cfg.finetune.local_update_steps = 20000000
#cfg.train.local_update_steps = 1

cfg.federate.total_round_num = 2000
cfg.model.dropout = 0.0
cfg.federate.client_num = 1
cfg.early_stop.patience = 100
#cfg.train.optimizer.lr = 0.001
#cfg.train.optimizer.weight_decay = 0.0005
#cfg.grad.grad_clip = 2.0
cfg.criterion.type = 'CrossEntropyLoss'
#cfg.seed = 123
cfg.eval.freq = 1
cfg.eval.metrics = ['imp_ratio']
cfg.eval.report = ['avg']
cfg.eval.best_res_update_round_wise_key = 'val_imp_ratio'
cfg.eval.count_flops = False


In [6]:
import torch
#torch.manual_seed(0)
#torch.backends.cudnn.benchmark = False
#torch.use_deterministic_algorithms(F/home/michael/Desktop/backup files/FederatedScope/data/CIKM22Competitionalse)
#import random
#random.seed(0)
#import numpy as np
#np.random.seed(0)

In [7]:
from yacs.config import CfgNode
client_cfg_file = "scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client_theirs.yaml"# "scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client_ours_lr_local_steps.yaml"
client_cfg = CfgNode.load_cfg(open(client_cfg_file,
                                       'r')) if client_cfg_file else None


### Start the FL prosess

In [8]:
from federatedscope.core.auxiliaries.data_builder import get_data
from federatedscope.core.auxiliaries.utils import setup_seed, update_logger
from federatedscope.core.fed_runner import FedRunner
from federatedscope.core.auxiliaries.worker_builder import get_server_cls, get_client_cls


2023-01-04 11:43:23,417 (trainer_builder:11)WARNING: No module named 'federatedscope.contrib.optimizer' in `federatedscope.contrib.trainer`, some modules are not available.


In [9]:
from federatedscope.contrib.workers.laplacian_with_domain_separation_MI_client import LaplacianDomainSeparationMIClient
from federatedscope.contrib.workers.laplacian_server_dom_sep import LaplacianServerDomSep

setup_seed(cfg.seed)
update_logger(cfg)
data, modified_cfg = get_data(cfg)
cfg.merge_from_other_cfg(modified_cfg)


2023-01-04 11:43:23,551 (utils:129)INFO: the current machine is at 127.0.1.1
2023-01-04 11:43:23,552 (utils:131)INFO: the current dir is /home/michael/Master-Thesis/CKIM_Competition
2023-01-04 11:43:23,552 (utils:132)INFO: the output dir is exp/test_dir/Laplacian_domain_sep_sum_MI_csd100.0_lam1_diff1_MI_lr_0.02_no_finetune_gin_on_graph-dt_lr0.1_lstep1_/sub_exp_20230104114323


In [10]:
Fed_runner = FedRunner(data=data,
                       server_class=LaplacianServerDomSep,
                       client_class=LaplacianDomainSeparationMIClient,
                       config=cfg.clone(),
                       client_config=client_cfg)
Fed_runner.run()

rho: 0.0
server params: 
encoder_atom.atom_embedding_list.0.weight


/home/michael/Master-Thesis/CKIM_Competition/venv/lib/python3.9/site-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
2023-01-04 11:43:25,730 (fed_runner:249)INFO: Server #0 has been set up ... 
2023-01-04 11:43:25,740 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 1
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criterion:
  type: MSELoss
data:
  args: []
  batch_size: 64
  cSBM_phi: [0.5, 0.5, 0.5]
  consistent_label_distribution: False
  drop_last: False
  graphsaint:
    num_steps: 30
    walk_length: 2
  loader: 
  num_workers: 0
  pre_transform: []
  quadratic:
    dim: 1
    max_curv: 12.5
  

encoder_atom.atom_embedding_list.1.weight
encoder_atom.atom_embedding_list.2.weight
encoder_atom.atom_embedding_list.3.weight
encoder_atom.atom_embedding_list.4.weight
encoder_atom.atom_embedding_list.5.weight
encoder_atom.atom_embedding_list.6.weight
encoder_atom.atom_embedding_list.7.weight
encoder_atom.atom_embedding_list.8.weight
encoder_atom.atom_embedding_list.9.weight
encoder_atom.atom_embedding_list.10.weight
encoder_atom.atom_embedding_list.11.weight
encoder_atom.atom_embedding_list.12.weight
encoder_atom.atom_embedding_list.13.weight
encoder_atom.atom_embedding_list.14.weight
encoder_atom.atom_embedding_list.15.weight
encoder_atom.atom_embedding_list.16.weight
encoder_atom.atom_embedding_list.17.weight
encoder_atom.atom_embedding_list.18.weight
encoder_atom.atom_embedding_list.19.weight
encoder_atom.atom_embedding_list.20.weight
encoder_atom.atom_embedding_list.21.weight
encoder.weight
encoder.bias
local_gnn.convs.0.nn.linears.0.weight
local_gnn.convs.0.nn.linears.0.bias
loca

2023-01-04 11:43:28,072 (laplacian_with_domain_separation_MI_client:140)INFO: {'Role': 'Client #1', 'Round': 0, 'Results_raw': {'train_imp_ratio': -1063.961928, 'train_total': 901, 'train_loss': 9693.408445, 'train_avg_loss': 10.7585}}


1: 6.95490837097168
2: 5.021333694458008
3: 22.20916748046875
4: 6.279468536376953
5: 12.830018997192383
6: 5.966901779174805
total: 59.26179885864258
1: 8.272886276245117
2: 2.954244613647461
3: 41.04423522949219
4: 28.499841690063477
5: 12.183189392089844
6: 9.30333137512207
total: 102.25772857666016


...

2023-01-04 11:43:31,494 (laplacian_with_domain_separation_MI_client:140)INFO: {'Role': 'Client #1', 'Round': 2, 'Results_raw': {'train_imp_ratio': -282.548246, 'train_total': 901, 'train_loss': 3185.840079, 'train_avg_loss': 3.535894}}
2023-01-04 11:43:31,496 (server:480)INFO: {'Role': 'Server #', 'Round': 2, 'Results_avg': {'test_imp_ratio': -385.406401, 'test_total': 113.0, 'test_loss': 506.987023, 'test_avg_loss': 4.486611, 'val_imp_ratio': -327.726387, 'val_total': 113.0, 'val_loss': 446.742698, 'val_avg_loss': 3.953475}}
2023-01-04 11:43:31,497 (monitor:513)INFO: current_best=-813.48153, should_save=False
2023-01-04 11:43:31,497 (monitor:513)INFO: current_best=-327.726387, should_save=True


1: 8.903264999389648
2: 3.817319869995117
3: 19.641637802124023
4: 6.108283996582031
5: 14.59503173828125
6: 6.819009780883789
total: 59.88454818725586
1: 7.903814315795898
2: 5.561351776123047
3: 17.93837547302246
4: 6.165742874145508
5: 14.317989349365234
6: 5.590200424194336
total: 57.477474212646484
1: 6.879091262817383
2: 2.5501251220703125
3: 18.274307250976562
4: 6.34455680847168
5: 16.241073608398438
6: 5.70368766784668
total: 55.992841720581055


2023-01-04 11:43:31,554 (laplacian_server_dom_sep:168)INFO: Server #0: Starting evaluation at the end of round 2.
2023-01-04 11:43:31,556 (laplacian_server_dom_sep:175)INFO: ----------- Starting a new training round (Round #3) -------------
2023-01-04 11:43:31,659 (client:410)INFO: {'Role': 'Client #1', 'Round': 3, 'Results_raw': {'test_imp_ratio': -269.104823, 'test_total': 113, 'test_loss': 385.514864, 'test_avg_loss': 3.411636, 'val_imp_ratio': -197.412539, 'val_total': 113, 'val_loss': 310.635197, 'val_avg_loss': 2.748984}}
2023-01-04 11:43:31,660 (monitor:513)INFO: current_best=-197.412539, should_save=True
2023-01-04 11:43:31,660 (client:431)INFO: Client: #1, val_imp_ratio: -197.412539. model saved at exp/test_dir/Laplacian_domain_sep_sum_MI_csd100.0_lam1_diff1_MI_lr_0.02_no_finetune_gin_on_graph-dt_lr0.1_lstep1_/sub_exp_20230104114323/model1.pth


Time for aggregation 1: 6.763458251953125
Time for aggregation 2: 46.863555908203125
1: 6.473779678344727
2: 2.5336742401123047
3: 24.34062957763672
4: 13.5650634765625
5: 14.054059982299805
6: 5.805730819702148
total: 66.7729377746582
1: 9.196758270263672
2: 5.326986312866211
3: 17.629146575927734
4: 11.347293853759766
5: 12.817144393920898
6: 7.0648193359375
total: 63.38214874267578


KeyboardInterrupt: 

-+/896*-Difference train_avg_loss: 125 -> 174
1: 0.539109 -> 0.540
2: 0.614181 -> 0.60414
3: 0.680621 -> 0.675371
4: 0.466614 -> 0.405735
5: 0.64907 -> 0.648837
6: 0.479263 -> 0.470293
7: 0.668663 -> 0.662036
8: 0.438008 -> 0.433998
9: 0.107733 -> 0.118945
10: 0.009074 -> 0.009043
11: 1.128238 -> 1.126667
12: 2.657362 -> 2.661828
13: -> 0.007134



+

{1: {}}

In [13]:
data

{1: {'train': <torch_geometric.loader.dataloader.DataLoader at 0x7f4ebc607190>,
  'val': <torch_geometric.loader.dataloader.DataLoader at 0x7f4de953c910>,
  'test': <torch_geometric.loader.dataloader.DataLoader at 0x7f4e247e3970>,
  'num_label': 0},
 2: {'train': <torch_geometric.loader.dataloader.DataLoader at 0x7f4ebc607190>,
  'val': <torch_geometric.loader.dataloader.DataLoader at 0x7f4de953c910>,
  'test': <torch_geometric.loader.dataloader.DataLoader at 0x7f4e247e3970>,
  'num_label': 0}}

In [ ]:
import os

,,0After 10 rounds:

diff: 0, csd: 0:
server: -8
c1 t/val: 0.375 /0.37
c2 t/val: 0.678 /0.606
c3 t/val: 0.699 /0.608
c4 t/val: 0.599 /0.537

diff: 0, csd: 0:
server: -4.5
c1 t/val: 0.372879 /0.328
c2 t/val: 0.678992 /0.605809
c3 t/val: 0.70166 /0.608609
c4 t/val: 0.597455 /0.534253



diff: 10, csd: 0:
server: -35
c1 t/val: 0.482/0.52
c2 t/val: 0.694/0.65
c3 t/val: 0.724/0.65
c4 t/val: 0.674/0.575

diff: 1, csd: 0:
server: -9.6
c1 t/val: 0.378194/0.319349
c2 t/val: 0.67461/0.609163
c3 t/val: 0.705559/0.615019
c4 t/val: 0.605581/0.572086

diff: 0, csd: 1e4:
server: -18
c1 t/val: 0.280962/0.23074
c2 t/val: 0.682652/0.603175
c3 t/val: 0.699094/0.67027
c4 t/val: 0.594985/0.609611

diff: 0 csd: 1e8
server: -18
c1 t/val: 0.2606/0.348(0.197)
c2 t/val: 0.653/0.0.602
c3 t/val: 0.691/0.597
c4 t/val: 0.581/0.629

diff: 0 csd: 1e8
server: -12.7
c1 t/val: 0.262647/0.249875
c2 t/val: 0.649273/0.0.600435
c3 t/val: 0.696132/0.615158
c4 t/val: 0.586597/0.656992


diff: 0.1 csd: 0
server: -11
c1 t/val: 0.2606/0.332837
c2 t/val: 0.678326/0.0.610099
c3 t/val: 0.704083/0.616179
c4 t/val: 0.604177/0.559658




In [4]:
os.environ['CUBLAS_WORKSPACE_CONFIG']

NameError: name 'os' is not defined